### **Importando bibliotecas**

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

### **"Arrumando" o dataset para o treinamento**

In [ ]:
batch_size = 128
num_classes = 10
epochs = 12

# Dimensões das imagens
img_rows, img_cols = 28, 28

# Carrega o dataset do mnist, embaralha os dados
# e separa em conjunto de treinamento e teste
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Arruma os dados para o treinamento de acordo com a configuração do backend
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

    
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normaliza os dados para serem 0 ou 1
# Fiz um teste sem normalizar e não senti diferença para esse dataset
x_train /= 255
x_test /= 255

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Converte os alvos para serem vetores 10x1 em que somente o neurônio
# corresponde ao alvo é ativado.
# Ex: Entrada é o número 5 -> Elemento 5 do vetor de saída vale "1" e os outros "0"
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

### **Montagem da rede e treinamento**

In [ ]:
# É montado o modelo da rede, camada a camada. Nesse caso, uma CNN
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# "Compila" o modelo com base na função custo e
# na otimização escolhida
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# Ajuste do modelo
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
          verbose=1, validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])